In [ ]:
!pip install earthengine-api geemap --upgrade

In [35]:
import ee

ee.Initialize(project="ee-romdijoux")

# Collect Vertical Transmit-Vertical Receive (VV) and Vertical Transmit-Horizontal Receive (VH) bands from Sentinel-1 at 10m resolution.
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD#description
sentinel1 = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VH"))
    .filter(ee.Filter.eq("resolution_meters", 10))
    .select(["VV", "VH"])
)

# Load Sentinel-2 Level 1C data, only RGB bands (B4, B3, and B2) captured at 10m resolution
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2#description
sentinel2 = ee.ImageCollection("COPERNICUS/S2").select(["B4", "B3", "B2"])


In [77]:
def sampleSentinelData(location, date):
    start_date = date.advance(-15, "days")
    end_date = date.advance(15, "days")

    print("Location : ", location.getInfo())

    sentinel1_image = (
        sentinel1.filterDate(start_date, end_date).filterBounds(location).first()
    )
    sentinel2_image = (
        sentinel2.filterDate(start_date, end_date).filterBounds(sentinel1_image.geometry().centroid()).first()
    )

    sentinel1_image_clipped = sentinel1_image.clip(sentinel2_image.geometry())
    sentinel2_image_clipped = sentinel2_image.clip(sentinel1_image_clipped.geometry())

    return (sentinel1_image_clipped, sentinel2_image_clipped)

In [78]:
from datetime import datetime, timedelta

## TODO: add more location, this is returing locations mostly in France but can be in the ocean or in neighboring countries too
def sampleCoord():
    return [random.uniform(-5.0, 9.0), random.uniform(42.0, 51.0)]


def sampleDate(start_date, end_date):
    start_datetime = datetime.strptime(start_date, '%Y-%m-%d')
    end_datetime = datetime.strptime(end_date, '%Y-%m-%d')
    date_range = (end_datetime - start_datetime).days
    random_days = random.randint(0, date_range)
    result_date = start_datetime + timedelta(days=random_days)
    return result_date.strftime('%Y-%m-%d')


In [79]:
import random
import geemap

start_date = "2018-01-01"
end_date = "2022-01-01"

# Generate a random location within ~ France
location  = sampleCoord()
date = sampleDate(start_date, end_date)

(sentinel1_image, sentinel2_image) = sampleSentinelData(
    ee.Geometry.Point(location), ee.Date(date)
)

print("Sentinel-1 Image:", sentinel1_image.getInfo())
print("Sentinel-2 Image:", sentinel2_image.getInfo())

cloud_percent = sentinel2_image.getInfo()["properties"]["CLOUD_COVERAGE_ASSESSMENT"]
print(cloud_percent, "percents of cloud coverage")

m = geemap.Map()
m.add_layer(sentinel1_image, {"min": -25, "max": 5}, "Sentinel-1", True)
m.add_layer(
    sentinel2_image,
    {"bands": ["B4", "B3", "B2"], "min": 0, "max": 65535, "gamma": 4.0},
    "Sentinel-2",
    True,
)

m

Location :  {'type': 'Point', 'coordinates': [3.636539403708367, 48.172182767187756]}
Sentinel-1 Image: {'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [10981, 10981], 'origin': [4690, 3141], 'crs': 'EPSG:32631', 'crs_transform': [10, 0, 353048.90363419283, 0, -10, 5531446.667886629]}, {'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [10981, 10981], 'origin': [4690, 3141], 'crs': 'EPSG:32631', 'crs_transform': [10, 0, 353048.90363419283, 0, -10, 5531446.667886629]}], 'version': 1704641070746501, 'id': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20190917T055917_20190917T055942_029057_034C21_D3E4', 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[1.6415267861124585, 48.657111198157146], [1.6415437860794122, 48.65711065044006], [3.132279975657313, 48.66504542524076], [3.1323428323677827, 48.66508186147119], [3.1324121140857897, 48.66511291516877], [3.1324164270445363, 48.

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…